In [16]:
import geopandas as gp
import pandas as pd
import uuid
import pickle

raw_data= pd.read_csv("../artifacts/data/raw/raw_bronze_format.csv")
data = gp.GeoDataFrame(raw_data[['geometry', 'ndvi_fn', 'ndvi_value']])
data = data.pivot_table(values=['ndvi_value'], index=['geometry'], columns=['ndvi_fn'], aggfunc='first')
data = data.droplevel(0, axis=1).rename_axis(None, axis=1)
other_cols = raw_data.groupby('geometry')[['crop_name', 'date', 'district', 'tehsil', 'state']].first()
data = data.merge(other_cols, left_index=True, right_index=True, how='inner').reset_index()
feature_order = ['geometry', 'date', 'state', 'district', "tehsil", 'oct_1f', 'oct_2f', 'nov_1f', 'nov_2f', 'dec_1f', 
                 'dec_2f', 'jan_1f', 'jan_2f', 'feb_1f', 'feb_2f', 'mar_1f', 'mar_2f', 'apr_1f', 'apr_2f', 'crop_name']

data = data[feature_order]

data.rename(columns={'crop_name':'crop_type'}, inplace=True)

In [17]:
data

,geometry,date,state,district,tehsil,oct_1f,oct_2f,nov_1f,nov_2f,dec_1f,dec_2f,jan_1f,jan_2f,feb_1f,feb_2f,mar_1f,mar_2f,apr_1f,apr_2f,crop_type
0,POINT (74.2447066 22.5899787),2022-10-01,Madhya Pradesh,Alirajpur,Bhavara,189.0,191.0,173.0,126.0,132.0,171.0,183.0,187.0,184.0,180.0,166.0,130.0,120.0,124.0,Wheat
1,POINT (74.3092005 22.3536019),2022-10-01,Madhya Pradesh,Alirajpur,Alirajpur,171.0,166.0,142.0,132.0,165.0,185.0,190.0,188.0,180.0,171.0,143.0,130.0,123.0,121.0,Wheat
2,POINT (74.3244481 22.4485955),2022-10-01,Madhya Pradesh,Alirajpur,Alirajpur,166.0,147.0,129.0,131.0,159.0,188.0,197.0,192.0,185.0,172.0,121.0,121.0,116.0,118.0,Wheat
3,POINT (74.350949 22.4241502),2022-10-01,Madhya Pradesh,Alirajpur,Alirajpur,172.0,151.0,134.0,132.0,177.0,192.0,197.0,195.0,186.0,156.0,126.0,120.0,118.0,120.0,Wheat
4,POINT (74.360787 22.2489687),2022-10-01,Madhya Pradesh,Alirajpur,Alirajpur,157.0,156.0,139.0,133.0,164.0,181.0,185.0,191.0,180.0,165.0,130.0,122.0,122.0,121.0,Wheat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29958,POLYGON ((83.80146126940511 25.608009064927664...,2022-10-01,Uttar Pradesh,Ghazipur,Muhammadbad,170.0,133.0,120.0,136.0,143.0,171.0,164.0,151.0,156.0,133.0,130.0,122.0,116.0,122.0,Potato
29959,POLYGON ((83.80296816034173 25.612416799565107...,2022-10-01,Uttar Pradesh,Ghazipur,Muhammadbad,162.0,138.0,119.0,139.0,151.0,168.0,159.0,138.0,128.0,124.0,118.0,121.0,125.0,126.0,Potato
29960,"POLYGON ((83.80337297341252 25.6097436602594, ...",2022-10-01,Uttar Pradesh,Ghazipur,Muhammadbad,153.0,137.0,123.0,141.0,156.0,177.0,169.0,154.0,155.0,119.0,118.0,125.0,127.0,134.0,Potato
29961,POLYGON ((83.80514678559226 25.605434517442173...,2022-10-01,Uttar Pradesh,Ghazipur,Muhammadbad,162.0,135.0,120.0,152.0,154.0,174.0,164.0,150.0,138.0,126.0,139.0,149.0,132.0,128.0,Potato


In [6]:
df.index = df.index.map(lambda x: uuid.uuid4())

with open('../artifacts/data/raw/master_pickle_raw.pkl', 'wb') as file:
    pickle.dump(df.to_dict(), file)

In [7]:
import pickle


with open('../artifacts/data/raw/master_pickle.pkl', 'rb') as file:
    raw_data_path = pickle.load(file)

In [15]:
df = pd.DataFrame(raw_data_path)

df.index = pd.DataFrame(raw_data_path).reset_index()['index'].apply(lambda row:str(row))

with open('../artifacts/data/raw/master_pickle.pkl', 'wb') as file:
    pickle.dump(df.to_dict(), file)

In [9]:
import geopandas as gp

raw_data = gp.GeoDataFrame(raw_data_path)

In [15]:
from copy import deepcopy

def dip_impute(row):
    lst = deepcopy(row)
    act_strt_idx = lst.index.get_loc('oct_2f')
    for i in range(len(lst.loc['oct_2f':'apr_1f'])):
        actual_idx = i + act_strt_idx
        if ((lst.iloc[actual_idx-1] - lst.iloc[actual_idx]) >= 20) and ((lst.iloc[actual_idx+1] - lst.iloc[actual_idx]) >= 20):
            lst.iloc[actual_idx] = (lst.iloc[actual_idx-1] + lst.iloc[actual_idx+1])/2
    return lst

raw_data.iloc[:2,5:-2].apply(dip_impute, axis=1)

,oct_1f,oct_2f,nov_1f,nov_2f,dec_1f,dec_2f,jan_1f,jan_2f,feb_1f,feb_2f,mar_1f,mar_2f,apr_1f
4a761a4a-38cc-4f98-a3a4-1cb7246bc365,189.0,191.0,173.0,126.0,132.0,171.0,183.0,187.0,184.0,180.0,166.0,130.0,120.0
9c842300-c23f-4484-a3aa-a5c8f08f8f06,171.0,166.0,142.0,132.0,165.0,185.0,190.0,188.0,180.0,171.0,143.0,130.0,123.0


In [19]:
row = raw_data.iloc[0,5:-2]

row.iloc[1] = 'gg'

In [18]:
kr.items()

dict_items([('krih', [4, 5]), ('hg', [7, 6])])

In [7]:
kr = {'krih':[4, 5], 'hg':[7, 6]}
lst = map(lambda x: (x[0], sum(x[1])), kr.items())
{k: v for k, v in lst}

import yaml

yaml.safe_load(open('../configs/experiment.yaml', 'r'))

{'defaults': [{'train': 'modelling'}, '_self_'],
 'modelling_params': {'bool': False}}

In [13]:
list(kr.items())[0][]

('krih', [4, 5])